In [101]:
import numpy as np
import pandas as pd
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [102]:
#what does the data look like

train_df.iloc[:1,:]


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1


In [103]:
# what do disaster tweets looke like

disasters = train_df[train_df["target"]==1]
disasters.iloc[:2,:]

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1


In [104]:
#what do non disaster tweets look like
print ("non_disaster tweet")
non_disasters = train_df[train_df["target"]==0]
non_disasters.iloc[:2,:]

non_disaster tweet


,id,keyword,location,text,target
15,23,NaN,NaN,What's up man?,0
16,24,NaN,NaN,I love fruits,0


In [105]:

count_vectorizer = feature_extraction.text.CountVectorizer()

In [106]:
#what if there are different words in train and test set? Assume training set big enough

train_vectors = count_vectorizer.fit_transform(train_df["text"])
test_vectors = count_vectorizer.transform(test_df["text"])

In [107]:

#tfidf = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1, 2))
tfidf_transformer = TfidfTransformer()
train_tfidf = tfidf_transformer.fit_transform(train_vectors)
train_tfidf.shape


(7613, 21637)

In [108]:
from sklearn.naive_bayes import MultinomialNB
text_clf = MultinomialNB().fit(train_tfidf, train_df["target"])
train_df["target"]

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64

In [109]:
scores = model_selection.cross_val_score(text_clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores

array([0.66666667, 0.65609425, 0.72431507])

In [110]:
#scores = model_selection.cross_val_score(tfidf, train_vectors, train_df["target"], cv=3, scoring="f1")
#scores
predicted = text_clf.predict(test_vectors)


In [111]:
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission["target"] = text_clf.predict(test_vectors)
sample_submission.head()

,id,target
0,0,1
1,2,0
2,3,1
3,9,1
4,11,1


In [98]:
sample_submission.to_csv("submission2.csv", index=False)